# InSituPy demonstration
Example dataset: https://www.10xgenomics.com/products/xenium-in-situ/preview-dataset-human-breast

In [1]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

In [2]:
from xeniumdata import XeniumData
import xeniumdata
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from xeniumdata.utils.utils import decode_robust_series
from dask_image.imread import imread
from xeniumdata.images import deconvolve_he
import numpy as np
import scanpy as sc

## Load data

In [3]:
# input directories
data_dirs = [
    Path("C:/Users/ge37voy/data/2301_CRC/2301-01_20230504/01_xenium_data/output-XETG00050__0003621__Region_1__20230504__121954"),
    Path("C:/Users/ge37voy/data/datasets/Xenium_FFPE_Human_Breast_Cancer_Rep1_outs/output-XETG00000__slide_id__sample_id/"),
    Path("C:/Users/ge37voy/data/20230811__134602__2314_OTCs_Wollenberg/output-XETG00050__0003555__OTC0304-1__20230811__134819")
]

In [4]:
img_dirs = [elem.parent / "unregistered_images" for elem in data_dirs]

In [5]:
i = 0
data_dir = data_dirs[i]
img_dir = img_dirs[i]

In [6]:
data_dir

WindowsPath('C:/Users/ge37voy/data/2301_CRC/2301-01_20230504/01_xenium_data/output-XETG00050__0003621__Region_1__20230504__121954')

In [7]:
xd = XeniumData(data_dir)

In [8]:
xd

XeniumData
Slide ID:	0003621
Region ID:	Region_1
Data path:	C:\Users\ge37voy\data\2301_CRC\2301-01_20230504\01_xenium_data
Data folder:	output-XETG00050__0003621__Region_1__20230504__121954
Metadata file:	experiment_modified.xenium

## Read different data modalities

In [9]:
xd.read_annotations()
xd.read_images()
xd.read_matrix()

### Show overview of data

In [10]:
xd

XeniumData
Slide ID:	0003621
Region ID:	Region_1
Data path:	C:\Users\ge37voy\data\2301_CRC\2301-01_20230504\01_xenium_data
Data folder:	output-XETG00050__0003621__Region_1__20230504__121954
Metadata file:	experiment_modified.xenium
    ➤ images
       nuclei:	(27352, 34139)
       HE:	(27352, 34139, 3)
       panCK:	(27352, 34139, 3)
    ➤ matrix
       AnnData object with n_obs × n_vars = 151305 × 280
	       obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
	       var: 'gene_ids', 'feature_types', 'genome'
	       obsm: 'spatial'
    ➤ annotations
       tanja:	41 annotations, 5 classes ('Tumor', 'Fat', 'Mucus', 'Immune cells', 'Peritumoral Zone') 

## View data interactively

In [11]:
xd.show()

Viewer(camera=Camera(center=(0.0, 2906.04375, 3627.1625), zoom=0.06652388246666312, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 5812.299999999999, 0.2125), (0.0, 7254.537499999999, 0.2125)), current_step=(13675, 17069), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'nuclei' at 0x21d54bfdbe0>, <Image layer 'HE' at 0x21d45c52f70>, <Image layer 'panCK' at 0x21d552d0df0>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x0000021D41215A60>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap

### Crop data

In [12]:
xx = xd.crop(shape_layer="Shapes", inplace=False)

napari.Viewer: napari


c:\Users\ge37voy\AppData\Local\miniconda3\envs\xparse\lib\site-packages\anndata\_core\anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [13]:
xx

XeniumData
Slide ID:	0003621
Region ID:	Region_1
Data path:	C:\Users\ge37voy\data\2301_CRC\2301-01_20230504\01_xenium_data
Data folder:	output-XETG00050__0003621__Region_1__20230504__121954
Metadata file:	experiment_modified.xenium
    ➤ images
       nuclei:	(3373, 4473)
       HE:	(3373, 4473, 3)
       panCK:	(3373, 4473, 3)
    ➤ matrix
       AnnData object with n_obs × n_vars = 5883 × 280
	       obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
	       var: 'gene_ids', 'feature_types', 'genome'
	       obsm: 'spatial'
    ➤ annotations
       tanja:	2 annotations, 1 classes ('Tumor',) 

In [14]:
xx.show()

Viewer(camera=Camera(center=(0.0, 358.27500000000003, 475.15000000000003), zoom=0.5077260951328888, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 716.7625, 0.2125), (0.0, 950.5125, 0.2125)), current_step=(1686, 2236), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'nuclei' at 0x21d3a921940>, <Image layer 'HE' at 0x21d62778160>, <Image layer 'panCK' at 0x21d62d5b0d0>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x0000021D41215A60>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

## Perform preprocessing steps

In [15]:
xx.normalize()
xx.hvg()
xx.reduce_dimensions(umap=True, tsne=False)

Store raw counts in anndata.layers['counts']...
Normalization, log-transformation...
Calculate highly-variable genes across all samples using seurat flavor...


c:\Users\ge37voy\AppData\Local\miniconda3\envs\xparse\lib\site-packages\scanpy\preprocessing\_normalization.py:196: UserWarning: Some cells have zero counts
  warn(UserWarning('Some cells have zero counts'))
c:\Users\ge37voy\AppData\Local\miniconda3\envs\xparse\lib\site-packages\scanpy\preprocessing\_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
c:\Users\ge37voy\AppData\Local\miniconda3\envs\xparse\lib\site-packages\anndata\_core\anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Dimensionality reduction...
Leiden clustering...


In [16]:
xx.show(keys=["leiden", "ACTA2", "LYZ"])

Viewer(camera=Camera(center=(0.0, 358.27500000000003, 475.15000000000003), zoom=0.5077260951328888, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 716.7625, 0.2125), (0.0, 950.6913406476865, 0.2125)), current_step=(1686, 2236), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'nuclei' at 0x21dc29fef40>, <Image layer 'HE' at 0x21dc8f11f10>, <Image layer 'panCK' at 0x21dc8ed7790>, <Points layer 'leiden' at 0x21dd6a23220>, <Points layer 'ACTA2' at 0x21dc2497dc0>, <Points layer 'LYZ' at 0x21dadb8bee0>], help='use <5> for transform, use <2> for add points, use <3> for select points', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[],

In [18]:
xx.matrix.X.toarray()

array([[0.        , 0.34317848, 0.        , ..., 0.        , 0.9699002 ,
        0.34317848],
       [0.        , 0.38582915, 0.        , ..., 0.        , 0.38582915,
        0.88066363],
       [0.        , 0.43475094, 0.24081777, ..., 0.        , 0.        ,
        0.43475094],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.8946168 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

## Save results

In [19]:
out_dir = data_dir.parent / "cropped_processed"

In [21]:
xx.save(out_dir, overwrite=True)